In [ ]:
class DoubleArrayTrieNode:
    def __init__(self, value=None):
        self.value = value
        self.base = 0
        self.check = 0
        self.fail = None

class DoubleArrayTrie:
    def __init__(self):
        self.nodes = [DoubleArrayTrieNode()]
        self.strings = []
        self.lengths = []

    def insert(self, string):
        self.strings.append(string)
        self.lengths.append(len(string))
        node = 0
        for i in range(len(string)):
            char = string[i]
            found = False
            for j in range(self.nodes[node].base, len(self.nodes)):
                if self.nodes[j].check == node and self.nodes[j].value == char:
                    node = j
                    found = True
                    break
            if not found:
                new_node = DoubleArrayTrieNode(char)
                self.nodes.append(new_node)
                self.nodes[node].base = len(self.nodes) - self.nodes[node].check
                node = len(self.nodes) - 1
        self.nodes[node].value = string

    def build(self):
        queue = []
        queue.append(0)
        while queue:
            parent = queue.pop(0)
            for i in range(self.nodes[parent].base, len(self.nodes)):
                if self.nodes[i].check == parent:
                    queue.append(i)
                    fail_node = self.nodes[parent].fail
                    while fail_node is not None and not self.nodes[fail_node].value:
                        fail_node = self.nodes[fail_node].fail
                    if fail_node is None:
                        self.nodes[i].fail = 0
                    else:
                        fail_child = self.nodes[fail_node].base + ord(self.nodes[i].value)
                        if fail_child < len(self.nodes) and self.nodes[fail_child].check == fail_node:
                            self.nodes[i].fail = fail_child
                        else:
                            self.nodes[i].fail = fail_node

    def search(self, string):
        node = 0
        for i in range(len(string)):
            char = string[i]
            found = False
            for j in range(self.nodes[node].base, len(self.nodes)):
                if self.nodes[j].check == node and self.nodes[j].value == char:
                    node = j
                    found = True
                    break
            if not found:
                while self.nodes[node].fail is not None:
                    node = self.nodes[node].fail
                    found = False
                    for j in range(self.nodes[node].base, len(self.nodes)):
                        if self.nodes[j].check == node and self.nodes[j].value == char:
                            node = j
                            found = True
                            break
                    if found:
                        break
            if not found:
                node = 0
            if self.nodes[node].value is not None and i == len(string) - 1:
                return self.nodes[node].value
        return None
    
trie = DoubleArrayTrie()
trie.insert("hello")
trie.insert("world")
trie.build()
result = trie.search("hello")
print(result) # 출력: hello
result = trie.search("world")
print(result) # 출력: world
result = trie.search("hell")
print(result) # 출력: None
result = trie.search("wor")
print(result) # 출력: None